## Script responsável pela remoção dos nome próprios

- Este código pressupõe que a existência do arquivo **.\saida\03_1_DOUSecao02_portarias_mgi_tratado_basico.parquet** 
- Caso não exista, ele pode ser gerado por meio do notebook **03_1_Tratamento_Basico.ipynb**
- Processamento deste script:
  - Faz a leitura do arquivo **./saida/03_1_DOUSecao02_portarias_mgi_tratado_basico.parquet**
  - Remove os nomes próprios da **coluna TextoTratado**
  - Salva a saída parcial no arquivo **.\saida\03_2_DouSecao02_portarias_mgi_tratado_NER.parquet** 
 
---


## Biblioteca

In [1]:
import pandas as pd
import spacy
import time

In [2]:
douItem = 2
secao = f'Secao0{douItem}'

# Lê o arquivo Parquet
df_portarias_mgi = pd.read_parquet(f'./saida/03_1_DOU{secao}_portarias_mgi_tratado_basico.parquet', engine='pyarrow')  # ou engine='fastparquet'

# Exibe as primeiras linhas do DataFrame
df_portarias_mgi.shape


(19258, 33)

In [3]:
df_portarias_mgi.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,SubTitulo,TextoHTML,file_name,zip_name,Ano,Mes,Texto,Total_palavras,TextoTratado,TextoTratado_TotalPalavras
0,30266027,PORTARIA DE PESSOAL 529,9368436,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,"<p class=""identifica"">PORTARIA DE PESSOAL SGP/...",S02012023/529_20230125_20280233.xml.xml,S02012023.zip,2023,1,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 529, DE 17...",358,"sgp/sedgg/me nº 529, 17 janeiro 2023 secretari...",231
1,30264410,ATO PORTARIA 655,9368530,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,"<p class=""identifica"">PORTARIA SGP/SEDGG/ME Nº...",S02012023/529_20230125_20280363.xml.xml,S02012023.zip,2023,1,"PORTARIA SGP/SEDGG/ME Nº 655, DE 23 DE JANEIRO...",245,"portaria sgp/sedgg/me nº 655, 23 janeiro 2023 ...",158
2,30265009,ATO PORTARIA DE PESSOAL 461,9368536,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,"<p class=""identifica"">PORTARIA DE PESSOAL SGP/...",S02012023/529_20230125_20280370.xml.xml,S02012023.zip,2023,1,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 461, DE 16...",251,"sgp/sedgg/me nº 461, 16 janeiro 2023 secretari...",160
3,30265678,ATO PORTARIA DE PESSOAL SGP 606,9368541,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,"<p class=""identifica"">PORTARIA DE PESSOAL SGP/...",S02012023/529_20230125_20280375.xml.xml,S02012023.zip,2023,1,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 606, DE 19...",251,"sgp/sedgg/me nº 606, 19 janeiro 2023 secretari...",158
4,30266030,ATO PORTARIA DE PESSOAL SGP 517,9368573,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,"<p class=""identifica"">PORTARIA DE PESSOAL SGP/...",S02012023/529_20230125_20280390.xml.xml,S02012023.zip,2023,1,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 517, DE 17...",243,"sgp/sedgg/me nº 517, 17 janeiro 2023 secretari...",158


In [4]:
# Carregar o modelo de linguagem em português.
nlp = spacy.load('pt_core_news_sm')

def extrair_nomes(texto):
    """
    Extrai nomes (entidades do tipo PER) do texto.
    Exclui nomes que começam com determinados prefixos, desconsiderando o case.
    """
    # Processar o texto com o modelo spaCy
    doc = nlp(texto)

    # Extrair entidades rotuladas como "PER" (Pessoas)
    nomes = [ent.text for ent in doc.ents if ent.label_ == 'PER']

    # Filtrar nomes para excluir aqueles que começam com os prefixos indesejados
    prefixos_indesejados = ('administracao', 'administrador', 'analista', 'anexo', 'cargo', 'apto', 'arquiteta', 'arquivista',
                        'arquivologia', 'arquivos', 'arts', 'artifice', 'ascensorista', 'assessor', 'assessora', 'assistente', 
                        'auditor', 'autoriza', 'auxiliar', 'bi', 'bancada', 'bancario', 'carteiro', 'carpinteiro', 
                        'cartorio', 'celetista', 'chefe', 'conceder', 'coordenador', 'coordenadora', 'coordenador-geral', 
                        'comissionada', 'dataprev', 'decreto', 'designar', 'diretor', 'diretoria', 
                        'escrivao', 'especializado', 'iii', 'inciso', 'instituidor', 'lei', 'matricula', 'paragrafo', 'patrimonio', 'professor',
                        'orgao', 'orgaos', 'siape', 'sisg', 'soldo', 'vii', 'xxx', 'uteis')

    nomes_filtrados = [
        nome for nome in nomes 
        if not nome.lower().startswith(prefixos_indesejados)
    ]

    # Retornar os nomes filtrados como uma única string, separados por vírgulas
    return ', '.join(nomes_filtrados)



# Função para remover nomes de pessoas
def remover_nomes(texto):
    doc = nlp(texto)
    # Excluir tokens que são identificados como nomes próprios (entidade do tipo 'PER')
    tokens_sem_nomes = [token.text for token in doc if token.ent_type_ != 'PER']
    return ' '.join(tokens_sem_nomes)

# Configurar o tamanho do bloco
tamanho_bloco = 200

# Inicializar colunas para os resultados
df_portarias_mgi['Texto_sem_nomes'] = None
df_portarias_mgi['Nomes'] = None

# Medir o tempo total de execução
inicio_tempo_total = time.time()

# Processar em blocos
for inicio in range(0, len(df_portarias_mgi), tamanho_bloco):
    fim = min(inicio + tamanho_bloco, len(df_portarias_mgi))  # Define o limite superior do bloco
    print(f"Tratando de {inicio + 1} a {fim}")
    
    # Selecionar o bloco atual
    bloco = df_portarias_mgi.iloc[inicio:fim]

    # Aplicar as funções ao bloco
    df_portarias_mgi.loc[inicio:fim - 1, 'TextoTratadoSemNomes'] = bloco['TextoTratado'].apply(remover_nomes)
    df_portarias_mgi.loc[inicio:fim - 1, 'Nomes'] = bloco['TextoTratado'].apply(extrair_nomes)

# Medir o tempo final
fim_tempo_total = time.time()

# Calcular o tempo total de execução
tempo_execucao_total = fim_tempo_total - inicio_tempo_total
print(f"Tempo total de execução: {tempo_execucao_total:.2f} segundos")


Tratando de 1 a 200
Tratando de 201 a 400
Tratando de 401 a 600
Tratando de 601 a 800
Tratando de 801 a 1000
Tratando de 1001 a 1200
Tratando de 1201 a 1400
Tratando de 1401 a 1600
Tratando de 1601 a 1800
Tratando de 1801 a 2000
Tratando de 2001 a 2200
Tratando de 2201 a 2400
Tratando de 2401 a 2600
Tratando de 2601 a 2800
Tratando de 2801 a 3000
Tratando de 3001 a 3200
Tratando de 3201 a 3400
Tratando de 3401 a 3600
Tratando de 3601 a 3800
Tratando de 3801 a 4000
Tratando de 4001 a 4200
Tratando de 4201 a 4400
Tratando de 4401 a 4600
Tratando de 4601 a 4800
Tratando de 4801 a 5000
Tratando de 5001 a 5200
Tratando de 5201 a 5400
Tratando de 5401 a 5600
Tratando de 5601 a 5800
Tratando de 5801 a 6000
Tratando de 6001 a 6200
Tratando de 6201 a 6400
Tratando de 6401 a 6600
Tratando de 6601 a 6800
Tratando de 6801 a 7000
Tratando de 7001 a 7200
Tratando de 7201 a 7400
Tratando de 7401 a 7600
Tratando de 7601 a 7800
Tratando de 7801 a 8000
Tratando de 8001 a 8200
Tratando de 8201 a 8400
Tra

In [5]:
df_portarias_mgi.shape

(19258, 36)

In [6]:
# Salva saída parcial o arquivo completo com novas colunas
df_portarias_mgi.to_parquet(f'./saida/03_2_DOU{secao}_portarias_mgi_tratado_NER.parquet', engine='pyarrow', index=False)